In [1]:
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import Column, Integer, String, BigInteger, VARCHAR, SmallInteger
from sqlalchemy import *
import requests
import csv
import os
from requests.auth import HTTPBasicAuth
from app_config import (
DIALECT,
DRIVER,
PORT,
HOST,
USER,
PASSWORD,
DATABASE
)

def to_dict(self):
    return {
        column.name: getattr(self, column.name)
        if not isinstance(getattr(self, column.name), datetime.datetime)
        else getattr(self, column.name).isoformat()
        for column in self.__table__.columns
    }

from models import Products

SQALCHEMY_DATABASE_URL = f"{DIALECT}+{DRIVER}://{USER}:{PASSWORD}@{HOST}/{DATABASE}"

engine = create_engine(
    SQALCHEMY_DATABASE_URL, 
    pool_recycle=3600,
    pool_pre_ping=True
)
Base = automap_base()
Base.prepare(engine, reflect=True)
Session = sessionmaker(bind=engine)
session = Session()


In [2]:
product_query = session.query(Products).all()


In [3]:
dict_list = [p.to_dict() for p in product_query]
product_names = [p['product_name'] for p in dict_list]

product_zip = zip(product_names, dict_list)


In [ ]:
filename ='Resources/product_nutrients.csv'

url = "https://api.edamam.com/api/food-database/parser"
data = []
for p in product_zip:
    try:
        querystring = {'ingr': p[0],
                   'app_id': '4ccefd3f',
                   'app_key': '5f95a55762f1cc4ed973d0c093d6ccad'
                  }
        response = requests.get(url, params=querystring).json()


        r_dict  = {
        'text_API': response['parsed'][0]['food']['label'],
        'ENERC_KCAL': response['parsed'][0]['food']['nutrients']['ENERC_KCAL'],
        'FAT': response['parsed'][0]['food']['nutrients']['FAT'],
        'CHOCDF': response['parsed'][0]['food']['nutrients']['CHOCDF'],
        'FIBTG': response['parsed'][0]['food']['nutrients']['FIBTG'],
        'PROCNT': response['parsed'][0]['food']['nutrients']['PROCNT']
        }

    except:
        pass
    
    
    full_data = p[1]
    if response:
        full_data.update(r_dict)
        data.append(full_data)
    else:
        full_data.update({
            'text_API': 0,
            'ENERC_KCAL': 0,
            'FAT': 0,
            'CHOCDF': 0,
            'FIBTG': 0,
            'PROCNT': 0
        })
        data.append(full_data)

    file_exists = os.path.isfile(filename)


    
    # writing to csv file  
    with open(filename, 'a', encoding='utf-8') as csvfile: 
        # creating a csv writer object
        fieldnames = full_data.keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        if not file_exists:
            writer.writeheader()

        # writing the data rows  
        writer.writerow(full_data)
    
        

    
    
    